In [105]:
import pandas as pd
import numpy as np

!mkdir ../data/preprocessed
!mkdir ../Algorithms

df = pd.read_csv("../data/start-data.csv")

mkdir: ../data/preprocessed: File exists
mkdir: ../Algorithms: File exists


# Data Preprocessing

## Feature Engineering

### Creating New Features From Category Names

In [106]:
cold_item_types = [
                    "Gazlı İçecek",
                    "Ayran & Kefir",
                    "Süt",
                    "Peynir",
                    "Migros Dondurma",
                    "Yoğurt",
                    "Tek Dondurma",
                    "Tereyağ & Margarin",
                    "Kap Dondurma",
                    "Meyve Suyu",
                    "Soğuk Kahve",
                    "Enerji İçeceği",
                    "Tereyağ",
                    "Şalgam"
                    ]

frozen_item_types = [
                    "Beyaz Et",
                    "Kırmızı Et",
                    "Et Şarküteri",
                    "Pizza",
                    "Mantı",
                    "Dondurulmuş Gıda",
                    "Sakatat",
                    "Donuk Fırın",
                    "Dondurulmuş",
                    "Balık",
                    "Buz"
                    ]

scalable_item_types = [
                      "Sebze",
                      "Meyve",
                      "Yeşillik",
                      ]

In [107]:
'''
# This function checks if any item types are present in the given category names.

# Arguments:
#   category_names (str): A comma-separated string containing category names.
#   item_types (iterable): An iterable collection of item types to search for.

# Returns:
#   int: The number of item types found in the category names.
'''
def contains_items(category_names, item_types):
    if isinstance(category_names, str):
        categories = category_names.split(',')
        item_count = 0
        for category in categories:
            if any(item.strip() in item_types for item in category.split(',')):
                item_count += 1
        return item_count
    else:
        return 0

In [108]:
# Apply the function to create the frozen_item column
df['frozen_item'] = df['category_names'].apply(lambda x: contains_items(x, frozen_item_types))
print("Frozen item column added to df")

# Apply the function to create the cold_item column
df['cold_item'] = df['category_names'].apply(lambda x: contains_items(x, cold_item_types))
print("Cold item column added to df")

# Apply the function to create the scalable_item column
df['scalable_item'] = df['category_names'].apply(lambda x: contains_items(x, scalable_item_types))
print("Scalable item column added to df")

Frozen item column added to df
Cold item column added to df
Scalable item column added to df


In [109]:
df

,packet_date,order_id,package_id,collection_duration,item_count,is_vip,category_names,units,cold_item,frozen_item,scalable_item
0,2023-01-30 16:25:56.531971 UTC,231986020,66538886,8.772191,21,False,"Meyve,Sebze,Sebze,Peynir,Zeytin,Bakliyat,Mutfa...","GRAM,GRAM,GRAM,GRAM,GRAM,PIECE,PIECE,PIECE,PIE...",3,0,6
1,2023-01-13 17:39:56.963085 UTC,226345108,64279854,17.366667,17,False,"Sebze,Konserve & Turşu,Meyve,Bakliyat,Meyve,Me...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",0,0,11
2,2023-12-16 13:39:03.726051 UTC,355151082,113977388,11.023730,17,False,"Ayran & Kefir,Kuruyemiş,Yumurta,Bar Çikolata,K...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",3,1,1
3,2023-02-11 13:37:56.262404 UTC,235698212,68008610,11.559015,19,False,"Kedi,Meyve,Kedi,Beyaz Et,Kraker,Bisküvi,Şekerl...","PIECE,GRAM,PIECE,GRAM,PIECE,PIECE,PIECE,PIECE,...",0,1,2
4,2023-01-04 10:36:28.288366 UTC,223149961,63047012,5.927209,12,False,"Sebze,Et Şarküteri,Sütlü Tatlı, Krema,Mutfak E...","GRAM,PIECE,PIECE,PIECE,PIECE,GRAM,GRAM,GRAM,PI...",0,2,4
...,...,...,...,...,...,...,...,...,...,...,...
1402792,2023-05-14 22:03:55.03007 UTC,267250768,80626839,7.512748,17,False,"Sebze,Kuruyemiş,Et Şarküteri,Meyve,Meyve,Sebze...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",0,1,8
1402793,2023-05-02 09:03:42.533162 UTC,262597520,78757438,10.733333,17,False,"Peynir,Meyve Suyu,Ekmek,Peynir,Peynir,Meyve Su...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",7,1,0
1402794,2023-02-13 15:23:39.449954 UTC,236659316,68303121,8.816667,17,False,"Et Şarküteri,Sebze,Çay,Peynir,Bulaşık,Kağıt,Ma...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",2,4,2
1402795,2023-03-26 17:25:06.829762 UTC,250406920,73772482,13.550000,17,False,"Unlu Mamüller,Sebze,Ekmek,Yeşillik,Bulaşık,Yeş...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",1,1,8


### Creating New Features From Packet Dates

In [110]:
# Convert the 'packet_date' column to datetime
df['packet_date'] = pd.to_datetime(df['packet_date'], format='%Y-%m-%d %H:%M:%S.%f %Z')

# Extract the month, day of the week, and hour from the 'packet_date' column
df['month'] = df['packet_date'].dt.month
df['day'] = df['packet_date'].dt.dayofweek
df['hour'] = df['packet_date'].dt.hour

print("Columns month, day, and hour added to df")

Columns month, day, and hour added to df


In [111]:
# check if the packet_date column contains holidays
import holidays

tr_holidays = holidays.Turkey()
df['is_holiday'] = df['packet_date'].apply(lambda x: 1 if x in tr_holidays else 0)
print("is_holiday column added to order_date.csv.")
#check how many holiday orders are there in the dataset
df.drop(columns=['packet_date'], inplace=True)
print("Number of holidays in the dataset: ", df['is_holiday'].sum())

is_holiday column added to order_date.csv.
Number of holidays in the dataset:  43967


In [112]:
print("Number of holidays in the dataset: ", df['is_vip'].sum())

Number of holidays in the dataset:  4274


In [113]:
df

,order_id,package_id,collection_duration,item_count,is_vip,category_names,units,cold_item,frozen_item,scalable_item,month,day,hour,is_holiday
0,231986020,66538886,8.772191,21,False,"Meyve,Sebze,Sebze,Peynir,Zeytin,Bakliyat,Mutfa...","GRAM,GRAM,GRAM,GRAM,GRAM,PIECE,PIECE,PIECE,PIE...",3,0,6,1,0,16,0
1,226345108,64279854,17.366667,17,False,"Sebze,Konserve & Turşu,Meyve,Bakliyat,Meyve,Me...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",0,0,11,1,4,17,0
2,355151082,113977388,11.023730,17,False,"Ayran & Kefir,Kuruyemiş,Yumurta,Bar Çikolata,K...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",3,1,1,12,5,13,0
3,235698212,68008610,11.559015,19,False,"Kedi,Meyve,Kedi,Beyaz Et,Kraker,Bisküvi,Şekerl...","PIECE,GRAM,PIECE,GRAM,PIECE,PIECE,PIECE,PIECE,...",0,1,2,2,5,13,0
4,223149961,63047012,5.927209,12,False,"Sebze,Et Şarküteri,Sütlü Tatlı, Krema,Mutfak E...","GRAM,PIECE,PIECE,PIECE,PIECE,GRAM,GRAM,GRAM,PI...",0,2,4,1,2,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402792,267250768,80626839,7.512748,17,False,"Sebze,Kuruyemiş,Et Şarküteri,Meyve,Meyve,Sebze...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",0,1,8,5,6,22,0
1402793,262597520,78757438,10.733333,17,False,"Peynir,Meyve Suyu,Ekmek,Peynir,Peynir,Meyve Su...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",7,1,0,5,1,9,0
1402794,236659316,68303121,8.816667,17,False,"Et Şarküteri,Sebze,Çay,Peynir,Bulaşık,Kağıt,Ma...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",2,4,2,2,0,15,0
1402795,250406920,73772482,13.550000,17,False,"Unlu Mamüller,Sebze,Ekmek,Yeşillik,Bulaşık,Yeş...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",1,1,8,3,6,17,0


## Data Cleaning

### Removing Outliers

In [114]:
# Get the number of rows before filtering
num_rows_before = df.shape[0]

# Below 1 min collection durations are not possible and above 30 min collection durations are not delivered.
df = df[(df['collection_duration'] > 1) & (df['collection_duration'] < 30)]

# Get the number of rows after filtering
num_rows_after = df.shape[0]

# Calculate the number of rows removed
num_rows_removed = num_rows_before - num_rows_after

print("Number of rows removed:", num_rows_removed)

Number of rows removed: 21873


Since the dataset is real life data, we need to remove it sensibly. So we will use the IQR method with multivariate outlier detection. As a Our target and our features.

In [115]:
"""
# This function removes outliers from a DataFrame (`data`) based on the Interquartile Range (IQR) method.

# Args:
#    data (pd.DataFrame): The input DataFrame containing the data.
#    group (list): A list of column names in `data` to be used for grouping. Outlier removal is done
#                  within each unique combination of group values.
#    column (str): The name of the column containing numerical data where outliers will be removed.

# Returns:
#    pd.DataFrame: A new DataFrame with outliers removed based on the IQR method within each group.
"""

def remove_outliers_iqr(data, group, column):
    # Group data by x and calculate IQR and outliers
    groups = data.groupby(group, observed=False)
    filtered_data = []
    
    for _, group_data in groups:
        Q1 = group_data[column].quantile(0.25)
        Q3 = group_data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Filter data within the bounds
        filtered_group_data = group_data[(group_data[column] >= lower_bound) & (group_data[column] <= upper_bound)]
        filtered_data.append(filtered_group_data)

    #Print the number of rows removed

    print("Number of rows removed:", len(data) - pd.concat(filtered_data).shape[0])

    return pd.concat(filtered_data)

# # Usage
df = remove_outliers_iqr(df, [
                              'cold_item', 'frozen_item',
                              'scalable_item', 'item_count',
                              'day','hour','month'
                             ],
                              'collection_duration')
# Randomize df
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df

,order_id,package_id,collection_duration,item_count,is_vip,category_names,units,cold_item,frozen_item,scalable_item,month,day,hour,is_holiday
0,222622472,62805438,1.650000,1,False,Kedi,PIECE,0,0,0,1,0,8,0
1,231844008,66462231,2.633333,1,False,Konserve & Turşu,PIECE,0,0,0,1,0,8,0
2,240641036,69959001,3.532285,1,False,Temizlik,PIECE,0,0,0,2,0,8,0
3,240639505,69958241,4.816667,1,False,Kedi,PIECE,0,0,0,2,0,8,0
4,238428810,69098317,1.523860,1,False,Bebek Beslenme,PIECE,0,0,0,2,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347201,253867587,75210104,22.850000,52,False,"Gofret,Peynir,Tablet Çikolata,Cips,Tek Dondurm...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",18,3,0,4,2,20,0
1347202,270974075,82182940,11.900000,50,False,"Çay,Süt,Süt,Kağıt,Meyve,Kahve,Kap Dondurma,Yoğ...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",18,3,3,5,3,23,0
1347203,293229415,90780403,17.100000,55,False,"Kuruyemiş,Bisküvi,Meyve Suyu,Tek Dondurma,Kuru...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",18,9,5,7,1,22,0
1347204,360792013,116198169,17.750000,58,False,"Peynir,Peynir,Baharat,Kağıt,Milföy,Sebze,Yeşil...","PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIECE,PIEC...",19,3,8,12,4,10,0


### Null and Duplicate Values

In [ ]:
# Check for null values in each column
null_counts = df.isnull().sum()

# Display the number of null values in each column
print("Number of null values in each column:")
print(null_counts)

# Check if there are any duplicates
num_duplicates = df.duplicated(subset=['order_id', 'package_id']).sum()
print("Number of duplicate rows:", num_duplicates)

Number of null values in each column:
order_id                 0
package_id               0
collection_duration      0
item_count               0
is_vip                   0
category_names         182
units                    0
cold_item                0
frozen_item              0
scalable_item            0
month                    0
day                      0
hour                     0
is_holiday               0
dtype: int64
Number of duplicate rows: 0


In [ ]:
# Check if there are any duplicates keep first for order & package id
df.drop_duplicates(subset=['order_id', 'package_id'], keep='first', inplace=True)
# Drop null values
df.dropna(inplace=True)
# Drop the columns that are not needed
df.drop(columns=['package_id','order_id','units'], inplace=True)

In [ ]:
df

,collection_duration,item_count,is_vip,category_names,cold_item,frozen_item,scalable_item,month,day,hour,is_holiday
0,1.650000,1,False,Kedi,0,0,0,1,0,8,0
1,2.633333,1,False,Konserve & Turşu,0,0,0,1,0,8,0
2,3.532285,1,False,Temizlik,0,0,0,2,0,8,0
3,4.816667,1,False,Kedi,0,0,0,2,0,8,0
4,1.523860,1,False,Bebek Beslenme,0,0,0,2,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...
1347201,22.850000,52,False,"Gofret,Peynir,Tablet Çikolata,Cips,Tek Dondurm...",18,3,0,4,2,20,0
1347202,11.900000,50,False,"Çay,Süt,Süt,Kağıt,Meyve,Kahve,Kap Dondurma,Yoğ...",18,3,3,5,3,23,0
1347203,17.100000,55,False,"Kuruyemiş,Bisküvi,Meyve Suyu,Tek Dondurma,Kuru...",18,9,5,7,1,22,0
1347204,17.750000,58,False,"Peynir,Peynir,Baharat,Kağıt,Milföy,Sebze,Yeşil...",19,3,8,12,4,10,0
